In [22]:
# Imports
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
from massimal import georeferencing, hyspec_io
import rasterio
from rasterio.warp import reproject, Resampling, calculate_default_transform, transform_bounds
from rasterio.transform import Affine
from rasterio.profiles import DefaultGTiffProfile
from rasterio.crs import CRS

In [2]:
# Paths
example_image_path = Path('/media/mha114/Massimal2/Massimal/Bodo_Sandsund/Hyperspectral/20210602/Area/3a_R_rs/Sandsund_May2021_Pika_L_17-Radiance From Raw Data-Reflectance from Radiance Data and Downwelling Irradiance Spectrum.bip.hdr')
example_image_stem = example_image_path.stem.split('.')[0]
lcf_path = example_image_path.parent / (example_image_stem + '.lcf')
times_path = example_image_path.parent / (example_image_stem + '.bip.times')
geotiff_dir = Path('/media/mha114/Massimal2/Massimal/Bodo_Sandsund/Hyperspectral/20210602/Area/3c_R_rs_GeoTIFF')
geotiff_path = geotiff_dir / (example_image_stem + '.tiff')
rgb_geotiff_path = geotiff_dir / (example_image_stem + '_rgb.tiff')
world_file_path = geotiff_dir / (example_image_stem + '.wld')

In [3]:
utm_epsg = georeferencing.world_file_from_lcf_times_files(lcf_path,times_path,world_file_path)

In [5]:
(image,wl,rgb_ind,metadata) = hyspec_io.load_envi_image(example_image_path)

In [9]:
image_rgb = image[:,:,rgb_ind]

In [7]:
#crs_str = 'EPSG:32633'
georeferencing.save_geotiff_with_affine_transform(
    image=image_rgb,
    crs_str=f'EPSG:{utm_epsg}',
    geotiff_path=rgb_geotiff_path,
    world_file_path=world_file_path,
    band_names=tuple([str(wl[band_ind]) for band_ind in rgb_ind])
)
# georeferencing.save_geotiff_with_affine_transform(
#     image=image,
#     crs_str=crs_str,
#     geotiff_path=geotiff_path,
#     world_file_path=world_file_path
# )

In [8]:
with rasterio.open(rgb_geotiff_path, "r") as dataset:
    print(dataset.descriptions)
    print(dataset.units)

('640.15', '550.07', '459.59')
(None, None, None)


In [ ]:
def save_geotiff_with_affine_transform(
        image,
        crs_str: str,
        geotiff_path,
        affine_transform: tuple = None,
        world_file_path = None,
        band_names: tuple[str] = None,
        channels_already_first: bool = False) -> None:
    """ Save georeferenced image (single- or multiband) as GeoTIFF using affine transform
    
    # Input arguments:
    image:
        2D or 3D NumPy array
    crs_str:
        Coordinate reference system mathcing the affine transform used.
        String accepted by the rasterio.crs.CRS.from_string() method
        (https://rasterio.readthedocs.io/en/latest/api/rasterio.crs.html#rasterio.crs.CRS.from_string)
        EPSG, PROJ, and WKT strings are accepted.
        Example: "EPSG:32633"
    geotiff_path:
        File path for the GeoTIFF file to be written.
        
    # Keyword arguments:
    affine_transform:
        A 6-parameter transform for translation, scaling and rotation of 
        the image. Example (for UTM 33N): 
        (0.025746, 0.025290, 500199.3122, 0.02402464, -0.02710314, 7455416.7654)
        The ordering corresponds to (A,B,C,D,E,F).
        Note that this is not the same ordering as in the world file.
        If both affine_transform and world_file_path are specified, or if none
        of them are specified, an error is thrown.
    world_file_path:
        Path to text file containing exactly 6 lines with a single
        numer on each line. See https://en.wikipedia.org/wiki/World_file
        for specification. Note that the ordering of the lines in the file
        corresponds to (A,D,B,E,C,F)
    channels_already_first:
        Flag indicating that the image is organized "channels-first".
        This is the format required by rasterio / GDAL when writing GeoTIFFs.
        If False (default), channels in image are assumed to be last 
        (shape (nrows,ncols,nchannels)), and the last axis will be moved
        to first position before the file is written.    
    """

    # Check affine transform input
    if (affine_transform is None): 
        if (world_file_path is None):
            raise ValueError("Please specify either affine_transform or world_file_path")
        else:
            (a,d,b,e,c,f) = np.loadtxt(world_file_path) # Load parameters in "world file order"
            affine_transform = (a,b,c,d,e,f)            # Use rasterio Affine ordering
    elif (world_file_path is not None):
            raise ValueError("Please specify either affine_transform or world_file_path (not both)")

    # Create affine transform object
    transform = Affine(*affine_transform)

    # Convert 2D array to 3D (for unified processing)
    if image.ndim < 3:
         image = np.atleast_3d(image)
         channels_already_first = False

    # Ensure "channels-first" array, as required by rasterio/GDAL
    if not channels_already_first:
        image = np.moveaxis(image,2,0)

    # Create default GeoTIFF profile and update with current parameters
    profile = DefaultGTiffProfile()
    profile.update(height = image.shape[1], 
                   width = image.shape[2],
                   count = image.shape[0], 
                   dtype = str(image.dtype),
                   crs = CRS.from_string(crs_str),
                   transform = transform)
    
    # Register GDAL format drivers and configuration options with a context manager.
    with rasterio.Env():
        # Write image with context manager
        with rasterio.open(geotiff_path, 'w', **profile) as dataset:
            if band_names is not None:
                for i in range(dataset.count):
                    dataset.set_band_description(i+1, band_names[i])
            dataset.write(image) 

In [20]:
utm_epsg

32633

In [21]:
transform, dims = calculate_default_transform(dst_crs=CRS.from_epsg(utm_epsg),height = image.shape[1], width = image.shape[2])

TypeError: calculate_default_transform() missing 1 required positional argument: 'src_crs'

In [ ]:
# left = 0
# bottom = image.shape[0]-1
# right = image.shape[1]-1
# top = 0

# output_bounds = transform_bounds()